In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

### Loading the data

In [2]:
NUM_SAMPLES = 10000

input_texts = []
target_texts = []
target_texts_input = []

In [3]:
t = 0

for line in open("../neural_machine_translation/data/hin.txt"):
    t += 1
    if t > NUM_SAMPLES:
        break
    
    if '\t' not in line:
        continue
        
    input_text, translation, *rest = line.rstrip().split('\t')
    
    target_text = translation + ' <eos>'
    target_text_input = '<sos> ' + translation
    
    input_texts.append(input_text)
    target_texts.append(target_text)
    target_texts_input.append(target_text_input)

In [4]:
print('Number of input texts:', len(input_texts))
input_texts[:15]

Number of input texts: 3061


['Wow!',
 'Duck!',
 'Duck!',
 'Help!',
 'Jump.',
 'Jump.',
 'Jump.',
 'Hello!',
 'Hello!',
 'Cheers!',
 'Cheers!',
 'Exhale.',
 'Exhale.',
 'Got it?',
 "I'm OK."]

In [5]:
print(target_texts_input[:15])
print(target_texts[:15])

['<sos> वाह!', '<sos> झुको!', '<sos> बतख़!', '<sos> बचाओ!', '<sos> उछलो.', '<sos> कूदो.', '<sos> छलांग.', '<sos> नमस्ते।', '<sos> नमस्कार।', '<sos> वाह-वाह!', '<sos> चियर्स!', '<sos> सांस छोड़।', '<sos> सांस छोड़ो।', '<sos> समझे कि नहीं?', '<sos> मैं ठीक हूँ।']
['वाह! <eos>', 'झुको! <eos>', 'बतख़! <eos>', 'बचाओ! <eos>', 'उछलो. <eos>', 'कूदो. <eos>', 'छलांग. <eos>', 'नमस्ते। <eos>', 'नमस्कार। <eos>', 'वाह-वाह! <eos>', 'चियर्स! <eos>', 'सांस छोड़। <eos>', 'सांस छोड़ो। <eos>', 'समझे कि नहीं? <eos>', 'मैं ठीक हूँ। <eos>']
